In [84]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [85]:
disasterCostData = "../ml/data/costDataRevised.csv"
cost_df = pd.read_csv(disasterCostData)
cost_df.head()

,Unnamed: 0,disasterNumber,obligationAmount,state,incidentType,declarationTitle,incidentBeginDate,lat,lon,incidentDate,incidentTime
0,0,3345,904490.14,WV,Severe Storms,SEVERE STORMS,2012-06-29T22:30:00.000Z,39.453524,-78.039552,2012-06-29,22:30:00.000Z
1,1,3346,842464.67,OH,Severe Storms,SEVERE STORMS,2012-06-29T22:00:00.000Z,38.839894,-83.505170,2012-06-29,22:00:00.000Z
2,2,3347,1409793.92,LA,Hurricane,TROPICAL STORM ISAAC,2012-08-26T00:00:00.000Z,30.211644,-90.918610,2012-08-26,00:00:00.000Z
3,3,3348,253189.73,MS,Hurricane,TROPICAL STORM ISAAC,2012-08-26T00:00:00.000Z,33.077787,-89.568066,2012-08-26,00:00:00.000Z
4,4,3349,0.00,MD,Hurricane,HURRICANE SANDY,2012-10-26T15:36:00.000Z,38.528853,-76.537764,2012-10-26,15:36:00.000Z


In [91]:
weather_data = []
for index, row in cost_df.iterrows():
    lat = row['lat'] 
    lon = row['lon']
    date = row['incidentDate']
    #lat = 39.4535243
    #lon = -78.0395522
    #date = '2012-06-29'
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lon}/{date}?unitGroup=us&key=WAH7SGPNNNY6MRXZ7KYSSKC8M&include=obs'
    response = requests.get(url)
    weather_data.append(response.json())


In [95]:
df = pd.json_normalize(
    weather_data, 
    record_path =['days'], 
    meta=['latitude','longitude','resolvedAddress']
)

In [107]:
disaster = cost_df['disasterNumber'].to_list()

In [109]:
df['disasterNumber'] = disaster

In [111]:
merged_weather_data = pd.merge(df, cost_df, how='inner',on='disasterNumber')

In [117]:
merged_weather_data_cleanded = merged_weather_data.drop(columns=['precipprob','preciptype','solarradiation','solarenergy','uvindex','Unnamed: 0'])

In [118]:
merged_weather_data.to_csv('../ml/data/disasterWeather.csv')